# SGA
This notebook demonstrates basic workflow on implementation of Simple Genetic Algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from core import *

np.set_printoptions(linewidth=np.nan)

Random initialization of bit strings of given length

In [ ]:
def bit_initializer(population_size: int, individual_length: int) -> np.ndarray:
    return np.random.rand(population_size, individual_length) < 0.5
    
# class Initializer(InitializerBase):
#     def __init__(self):
#         super(Initializer, self).__init__()

#     def __call__(self, population_size: int, **kwargs) -> np.ndarray:
#         raise NotImplementedError

Objective function

In [ ]:
def objective_fnc(genes: np.ndarray) -> np.ndarray:
    return genes.sum(-1)
    
# class ObjectiveFnc(ObjectiveFncBase):
#     def __init__(self):
#         super(ObjectiveFnc, self).__init__()
        
#     def __call__(self, individuals: np.ndarray) -> np.ndarray:
#         raise NotImplementedError

Fitness function

In [ ]:
# def fitness_fnc(individuals: np.ndarray, objectives: np.ndarray) -> np.ndarray:
#     raise NotImplementedError
    
# class FitnessFnc(FitnessFncBase):
#     def __init__(self):
#         super(FitnessFnc, self).__init__()
    
#     def __call__(self, individuals: np.ndarray, objectives: np.ndarray) -> np.ndarray:
#         raise NotImplementedError

Early stopping criterion

In [ ]:
def early_stopping(ga: GeneticAlgorithm):
    raise NotImplementedError

# class EarlyStopping(EarlyStoppingBase):
#     def __init__(self):
#         super(EarlyStopping, self).__init__()
        
#     def __call__(self, ga: GeneticAlgorithm) -> bool:
#         raise NotImplementedError

SGA operators as designed by Holland

In [ ]:
class RouletteWheelSelection(OperatorBase):
    def __init__(self, whole_op: OperatorBase):
        super(RouletteWheelSelection, self).__init__(whole_op)
        
    def _operation(self, ga: GeneticAlgorithm, whole: Population) -> Population:
        chosen_index = np.random.choice(whole.size, size=whole.size, p=(whole.fitnesses / whole.fitnesses.sum()))
        
        return Population(whole.genes[chosen_index], ga)

In [ ]:
class OnePointXover(OperatorBase):
    def __init__(self, parents_op: OperatorBase, prob: float = 0.7):
        super(OnePointXover, self).__init__(parents_op)
        
        self.prob = prob
        
    def _operation(self, ga: GeneticAlgorithm, parents: Population) -> Population:
        xover_index = np.arange(0, parents.size, 2)[np.random.rand(parents.size // 2) < self.prob]
        xover_edges = (np.random.rand(len(xover_index)) * parents.genes.shape[1]).astype(np.int)
        
        offspring_genes = parents.genes.copy()
        for x_i in range(len(xover_index)):
            p1_i , p2_i = xover_index[x_i], xover_index[x_i] + 1
            xover_edge = xover_edges[x_i]
            
            offspring_genes[p1_i, xover_edge:] = parents.genes[p2_i, xover_edge:]
            offspring_genes[p2_i, xover_edge:] = parents.genes[p1_i, xover_edge:]
        
        return Population(offspring_genes, ga)

In [ ]:
class BitFlipMutation(OperatorBase):
    def __init__(self, naturals_op: OperatorBase, individual_prob: float = 0.05, gene_prob: float = 0.05):
        super(BitFlipMutation, self).__init__(naturals_op)
        
        self.individual_prob = individual_prob
        self.gene_prob = gene_prob
        
    def _operation(self, ga: GeneticAlgorithm, naturals: Population) -> Population:
        mutation_index = np.arange(naturals.size)[np.random.rand(naturals.size) < self.individual_prob]
        mutation_mask = np.random.rand(len(mutation_index), naturals.genes.shape[1]) < self.gene_prob
        
        mutants_genes = naturals.genes.copy()
        for m_i in range(len(mutation_index)):
            i_i = mutation_index[m_i]
        
            mutants_genes[i_i, mutation_mask[m_i]] = ~naturals.genes[i_i, mutation_mask[m_i]]
        
        return Population(mutants_genes, ga)

Callbacks for visual population tracking, fitness tracking

In [ ]:
class PopulationVisualReport(CallbackBase):
    def __init__(self, ax: plt.Axes):
        super(PopulationVisualReport, self).__init__()
        
        self._ax = ax
        
    def __call__(self, ga: GeneticAlgorithm) -> None:
        self._ax.clear()
        self._ax.axis('off')
        self._ax.imshow(np.swapaxes(ga.capture(-1).genes, 0, 1))

class ObjectiveReport(CallbackBase):
    def __init__(self, ax: plt.Axes):
        super(ObjectiveReport, self).__init__()
        
        self._ax = ax
    
    def __call__(self, ga: GeneticAlgorithm) -> None:
        self._ax.clear()
        self._ax.plot(np.max(ga.objectives_history, axis=-1))
        self._ax.set_title(
            'Current best fitness: {:.6f}'.format(np.max(ga.objectives_history[-1]))
        )
        
class GAStatus(CallbackBase):
    def __init__(self, fig: plt.Figure):
        super(GAStatus, self).__init__()

        self._fig = fig

    def __call__(self, ga: GeneticAlgorithm) -> None:
        self._fig.canvas.set_window_title('Current generation: {}'.format(ga.current_generation))
        self._fig.canvas.draw()

In [ ]:
%matplotlib ipympl

fig, ax = plt.subplots(2, 1, figsize=(8,7))
fig.tight_layout()

pop_report = PopulationVisualReport(ax[0])
obj_report = ObjectiveReport(ax[1])
ga_status = GAStatus(fig)

In [ ]:
graph = OperatorGraph()

selection = RouletteWheelSelection(graph.init_op)
xover = OnePointXover(selection)
mutation = BitFlipMutation(xover)

ga = GeneticAlgorithm(
    initializer = bit_initializer,
    operator_graph = graph,
    objective_fnc = objective_fnc,
    fitness_fnc = None,
    callbacks = [ga_status, pop_report, obj_report]
)

In [ ]:
result, fitnesses, objectives = ga.run(
    population_size = 128,
    generation_cap = 64,
    individual_length = 32
)